In [1]:
JD = """
Mô tả công việc
LG CNS is looking for full stack developers for cloud domain project

Develop software/web applicant
Business Analyst
Cooperate between HQ and VNB

Yêu cầu ứng viên
[Required]

Bachelor's degree of Information Technology or higher
Have working experiment and excellent knowledge at software developing using Java, Spring boot
Have working experiment and excellent knowledge at software developing using React, HTML, JavaScript
Good knowledge about AI (Azure OpenAI and GenAI)
Good knowledge about public cloud (Azure, AWS,)
Database: MariaDB (or MySQL).
[Preferred]

Having experiment with Python and Google cloud
"""

In [2]:
CHAT_SYSTEM_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

You are interviewing interviewee. You will communicate directly with interviewee.
You will use function call get_qestion to get theory interview question to ask interviewee 
then receive answer from interviewee, you analyitc the answer, then use function call result to send your evaluate, score of the answer (a number from 0 to 10); then get instruction of next step you will do from return of function, you must do following instruction.
"""

In [3]:
QUESTION_GENERATE_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

Generate suitable question base on job description and chat history.
output only 1 question. output question content only without any explain.
"""

In [4]:
CODE_QUESTION_GENERATE_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

Generate suitable code question base on job description and chat history.
must more detail as possible, with detail description and with at least 5 testcase (input/output sample pair), 
without solving question, in hard medium level
output only 1 question. output question content only without any explain.
"""

In [5]:
EVALUATE_INSTRUCTION = f"""You are an interviewer for job with description:
{JD}

I will give you chat history. 
Evaluate interview progress base on job description and chat history.
Make sure cover all topic of job description.
Change to next topic if you evaluate interviewee fully answer question or interviewee wrong answer question.
Give choose suitable action following to do: change other topic, following current topic, fail interview, pass interview.
Output progress as a number from 0 to 100 represent progress of interview.
Output total score of interviewee as a number from 0 to 100.
Output analysis why choose the action
Only output action i listed.
Only output result json, mustn't output markdown
Output json in format: progress: "", action: "", score: "", analysis: ""
"""

In [6]:
import os, dotenv
import google.generativeai as genai
dotenv.load_dotenv(".env")
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

e:\School\HK1 2024-2025\ML\Project\pre-view\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def generate_qestion(chatHistory, action):
    model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=QUESTION_GENERATE_INSTRUCTION)
    response = model.generate_content(f"Generate {action} question base on job description and chat history: {chatHistory}")
    return response.text

In [8]:
def generate_code_qestion(chatHistory):
    model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=CODE_QUESTION_GENERATE_INSTRUCTION)
    response = model.generate_content(f"Generate code question base on job description and chat history: {chatHistory}")
    return response.text

In [9]:
import json
from pydantic import BaseModel
from enum import Enum
from google.generativeai import GenerationConfig

Retry = 5

class ActionTag(str, Enum):
    OtherTopic = "change other topic"
    CurrentTopic = "following current topic"
    PassInterview = "pass interview"
    FailInterview = "fail interview"

class EvaluationExtractedOutput(BaseModel):
    analysis: str
    action: ActionTag
    score: float
    progress: float

def generate_evaluate(chatHistory: str, qestion: str, answer: str):
    for i in range(Retry):
        try:
            model = genai.GenerativeModel("gemini-1.5-flash", system_instruction=EVALUATE_INSTRUCTION,
                                    generation_config=GenerationConfig(
                                        temperature=(i + 1) * 0.4, 
                                        response_schema=EvaluationExtractedOutput,
                                        response_mime_type="application/json"))
            response = model.generate_content(f"Evaluate interview progress base on job description and chat history: {chatHistory}\n Interviewer: {qestion}\n Interviewee: {answer}")
            #print(response.text)
            return EvaluationExtractedOutput(**json.loads(response.text))
        except Exception as e:
            print(e)
            pass
    return EvaluationExtractedOutput(action=ActionTag.CurrentTopic, analysis="", score=50, progress=0)

In [10]:
import numpy as np
import google.generativeai as genai


i = 0
chatHistory = ""
currentQuestion = ""
currentAnswer = ""
nextAction = ActionTag.CurrentTopic

def result(evaluate: str, score: float):
    print("- review:", evaluate)
    score = np.clip(score, 0, 10)
    print("- score:", score)
    global i
    global currentQuestion
    global currentAnswer
    global chatHistory
    global nextAction
    i += 1
    evl = generate_evaluate(chatHistory, currentQuestion, currentAnswer)
    print(evl)
    nextAction = evl.action
    """Send evaluate and score about the answer provided by interviewee and get instruction of next step.
    Args: 
        evaluate: your evaluate about the answer provided by interviewee.
        score: floating point number from 0 to 10 represent your score for the answer provided by interviewee.
    Returns: 
        A string that is next step you will do.
    """
    return """ask interviewee next theory question. Don't stop interview, continue interview progress. 
                Use function call get_qestion to get theory interview question to ask interviewee,
                then receive answer from interviewee, 
                you analyitc the answer then use function call result to send your evaluate, score of the answer (a number from 0 to 10), then get instruction of next step you will do from return of function. you must do following instruction.""" if evl.progress < 100 and evl.action != ActionTag.FailInterview and evl.action != ActionTag.PassInterview else "stop interview"


def get_qestion():
    global i
    global chatHistory
    global currentQuestion
    global nextAction
    currentQuestion = generate_qestion(chatHistory, nextAction.name) 
    #print(i)
    #print(chatHistory)
    """Get Qestion for interview

    Args:
        
    Returns:
        A string that is a qestion you will ask interviewee.
    """
    return currentQuestion

import time
class ChatSession:
    def __init__(self):
        self.model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                                    system_instruction=CHAT_SYSTEM_INSTRUCTION,
                                    tools=[get_qestion, result])
        self.chat = self.model.start_chat(enable_automatic_function_calling=True)
        self.last_send_time = time.time()

    def sendUserMessage(self, data: str):
        global currentAnswer
        global chatHistory
        currentAnswer = data
        try:
            time.sleep(time.time() - self.last_send_time - 60/15)
        except:
            pass
        response = self.chat.send_message(data)
        self.last_send_time = time.time()
        chatHistory += "Interviewee: " + data + "\n"
        chatHistory += "Interviewer: " + response.candidates[0].content.parts[0].text + "\n"
        return response.candidates[0].content.parts[0].text

In [11]:
chatSession = ChatSession()
chatSession.sendUserMessage("hello")

"Hello!  Let's start the interview.  I'll begin by asking you some theoretical questions to assess your understanding of the technologies mentioned in the job description.\n"

In [12]:
chatSession.sendUserMessage("ok")


'Okay, the first question is:  "Describe your experience using Java, Spring Boot, React, HTML, JavaScript to develop a software application, highlighting the challenges you faced and how you overcame them."  Please provide a detailed answer.\n'

In [13]:
chatSession.sendUserMessage("""I developed an inventory management system using **Java**, **Spring Boot**, **React**, **HTML**, and **JavaScript**. The backend managed product data and user authentication, while the React frontend handled real-time updates and dynamic dashboards.

One challenge was ensuring seamless synchronization between the frontend and backend for stock updates. Initial polling methods caused delays and inconsistencies. I resolved this by implementing **WebSocket communication** in Spring Boot, enabling real-time updates in React whenever stock levels changed.

Another issue was managing complex forms with dynamic validation on the frontend. I used React’s state management and custom hooks to streamline form handling, improving usability and reducing user errors. This project strengthened my skills in full-stack development and real-time systems.""")

- review: Good understanding of full-stack development and problem-solving.  The answer clearly articulates challenges and solutions, demonstrating practical experience.
- score: 8.0
analysis='The interviewee provided a good answer, showcasing experience with Java, Spring Boot, React, HTML, and JavaScript.  The description of challenges and solutions demonstrates problem-solving skills. However, more probing questions are needed to fully assess their depth of knowledge and to cover other aspects of the job description (AI, Cloud, Databases).' action=<ActionTag.CurrentTopic: 'following current topic'> score=75.0 progress=25.0


'Okay, the next question is: "Describe a project where you integrated AI capabilities (preferably using Azure OpenAI or GenAI) into a web application built using Java, Spring Boot, React, and a MariaDB database, detailing your approach to data management and model deployment on a public cloud platform (Azure or AWS)."\n'

In [14]:
chatSession.sendUserMessage("""In a project where I integrated AI into a customer support platform, I used **Azure OpenAI** to power a chatbot built with **Java**, **Spring Boot**, **React**, and **MariaDB**. The goal was to automate responses to customer queries based on previous interactions and knowledge base articles.

For **data management**, I stored user interactions and knowledge base content in **MariaDB**. This allowed the backend to fetch relevant data to enhance AI responses and maintain context. I implemented **Spring Data JPA** to interact with the database efficiently.

For **model deployment**, I used **Azure** to host the OpenAI API. The Spring Boot application communicated with the Azure OpenAI model through REST APIs. Data privacy was ensured by sanitizing user inputs before sending them to the AI model. I also set up **Azure Cognitive Services** to handle AI processing, with the application deployed on **Azure App Services** for easy scalability.

This integration enhanced the platform's functionality, offering users immediate, AI-powered support while maintaining efficient data storage and management.""")

- review: Excellent response demonstrating a comprehensive understanding of AI integration, data management, and cloud deployment.  The use of specific technologies and attention to data privacy are strong points.
- score: 9.0
analysis="The interviewee has demonstrated a good understanding and practical experience with Java, Spring Boot, React, HTML, JavaScript, and AI integration using Azure OpenAI.  They provided detailed explanations of their projects, highlighting challenges and solutions.  To ensure a comprehensive evaluation, it's necessary to move on to other topics like database management (MariaDB), public cloud experience (Azure/AWS), and potentially Python/Google Cloud if the candidate has experience in those areas. " action=<ActionTag.OtherTopic: 'change other topic'> score=85.0 progress=50.0


'Okay, our next question is: "Describe a situation where you had to collaborate effectively with colleagues or stakeholders from different cultural backgrounds or geographical locations, and how you navigated any communication or collaboration challenges."\n'

In [15]:
chatSession.sendUserMessage("""In a project to develop a global e-commerce platform, I collaborated with team members from the U.S., India, and Germany. The challenge was coordinating across time zones, and navigating cultural differences in communication styles—some preferred detailed, written documentation, while others leaned towards more direct, brief exchanges.

To address this, we used tools like **Slack** for real-time communication and **Jira** for task management. I made sure to schedule regular video calls during overlapping working hours to foster clearer discussions. Additionally, I adapted my communication to accommodate different preferences, providing concise updates for some and detailed reports for others.

By being mindful of time zone constraints and cultural differences, we successfully aligned on project goals and delivered the platform on time.""")

- review: Good answer showcasing awareness of cultural sensitivity and effective use of collaboration tools.  Mentioning specific tools and strategies enhances the response.
- score: 7.0
analysis='The interviewee has demonstrated a good understanding and practical experience with Java, Spring Boot, React, HTML, JavaScript, AI integration (Azure OpenAI), database management (MariaDB), and cloud platforms (Azure).  The responses are detailed and show problem-solving skills.  However, to fully assess the candidate, we need to explore other aspects of the job description, such as experience with Python and Google Cloud (preferred skills), and delve deeper into the business analysis and cross-cultural collaboration aspects.' action=<ActionTag.OtherTopic: 'change other topic'> score=85.0 progress=60.0


'Okay, the next question is: "Describe a time you had to make a difficult technical decision with limited information, and how you approached the problem."\n'

In [16]:
chatSession.sendUserMessage("""In a project where we were integrating a third-party payment gateway into our e-commerce platform, I faced a difficult decision with limited information on the API's reliability and performance. The documentation was minimal, and there were concerns about how the integration would impact transaction speed and security.

To address this, I first conducted a quick prototype to test the API's performance under different loads. I also consulted with colleagues who had experience with similar integrations. Given the risks, I chose to implement the gateway with an initial fallback mechanism, ensuring that if the third-party API failed, users would be redirected to an alternative payment method.

This decision allowed us to move forward while mitigating potential risks, and I continuously monitored the integration's performance to make adjustments as needed.""")

- review: A solid response demonstrating a pragmatic approach to decision-making under pressure. The emphasis on risk mitigation and continuous monitoring is commendable.
- score: 8.0
analysis="The interviewee has consistently provided detailed and relevant answers, demonstrating a strong understanding of Java, Spring Boot, React, HTML, JavaScript, AI integration (Azure OpenAI), database management (MariaDB), and cloud platforms (Azure).  They have also showcased experience in overcoming technical challenges, collaborating effectively in diverse teams, and making sound technical decisions under pressure.  All required and preferred skills are covered. The interviewee's responses meet the criteria of the job description." action=<ActionTag.PassInterview: 'pass interview'> score=95.0 progress=100.0


'Okay, final question: "Describe a situation where you had to debug a complex issue in a large, distributed system spanning Java, Spring Boot, React, and a MariaDB database, outlining your troubleshooting methodology and the tools you used."\n'